In [1]:
import numpy as np
import scipy.io
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from itertools import product
import random
from tqdm import trange
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Load and scale training data
train_mat = scipy.io.loadmat('Xtrain.mat')
train_data = train_mat['Xtrain'].flatten().reshape(-1, 1)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(train_data)

# Load real test data
test_mat = scipy.io.loadmat('Xtest.mat')  
print(test_mat.keys())  # Check available keys
test_data = test_mat['Xtest'].flatten()  # Replace 'Xtest' with actual key if different

In [4]:
# Load the saved file with the best model and its configuration
checkpoint = torch.load('best_model.pth')

# Extract saved data
model_state_dict = checkpoint['model_state_dict']
best_config = checkpoint['config']  # (n_steps, lr, hidden_sizes, drop_out)
best_train_loss = checkpoint['val_loss']

# Prepare forecasting seed
n_steps = best_config[0]  # n_steps from your best model config
seed_seq = list(scaled[-n_steps:].flatten())  # last window from training data

In [5]:
# - best_train_loss (MSE on scaled data)
# - scaler (MinMaxScaler used on target data)

# 1. Scaled metrics
mse_scaled = best_train_loss
rmse_scaled = mse_scaled ** 0.5

# 2. Unscaled metrics
range_ = scaler.data_max_ - scaler.data_min_
mse_unscaled = mse_scaled * (range_[0] ** 2)
rmse_unscaled = rmse_scaled * range_[0]

# 3. Print all
print(f"Scaled MSE:  {mse_scaled:.6f}")
print(f"Sclaed RMSE: {rmse_scaled:.6f}")
print(f"Real MSE:  {mse_unscaled:.6f}")
print(f"Real RMSE: {rmse_unscaled:.6f}")

Scaled MSE:  0.000162
Sclaed RMSE: 0.012724
Real MSE:  10.363048
Real RMSE: 3.219169


In [ ]:
# ----  Recursive Forecasting Function ----

def recursive_forecast(model, seed_seq, n_future, scaler):
    model.eval()
    forecast = []
    input_seq = seed_seq.copy()

    for _ in range(n_future):
        input_tensor = torch.tensor(input_seq, dtype=torch.float32).view(1, -1)
        with torch.no_grad():
            next_val = model(input_tensor).item()
        forecast.append(next_val)
        input_seq = input_seq[1:] + [next_val]  # Slide the window

    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    return forecast.flatten()

# Step 4: Forecast as many steps as in test set
n_future = len(test_data)
predictions = recursive_forecast(model_state_dict, seed_seq, n_future, scaler)

In [ ]:
# Step 5: Evaluate
mae = mean_absolute_error(test_data, predictions)
mse = mean_squared_error(test_data, predictions)

print(f"Training Loss: {best_train_loss}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")

In [ ]:
# Step 6: Prepare full plot range (seed + forecast)
seed_real = scaler.inverse_transform(np.array(seed_seq).reshape(-1, 1)).flatten()

# Step 7: Plot seed, predicted, and actual test data
plt.figure(figsize=(12, 5))
plt.plot(np.arange(len(seed_real)), seed_real, label='Seed Input (History)', color='gray')
plt.plot(np.arange(len(seed_real), len(seed_real) + n_future), test_data, label='Actual Test Data', color='blue')
plt.plot(np.arange(len(seed_real), len(seed_real) + n_future), predictions, label='Predicted Values', color='red', linestyle='--')
plt.title('Recursive Forecast vs Actual Test Data')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()